### Обработка данных 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import re

In [2]:
def caesar_cipher(text: str, k: int) -> str:
    encrypted_text = ""
    for s in text:
        order = new_order = ord(s)
        if s.isalpha():
            new_order = order + k
            if s.isupper():
                if new_order > ord("Я"):
                    new_order = ord('А') + order + k - ord('Я') - 1
            else:
                if new_order > ord("я"):
                    new_order = ord('а') + order + k - ord('я') - 1
        encrypted_text += chr(new_order)
    return encrypted_text

In [3]:
CHARS = list('абвгдежзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [4]:
# все строки списками из индексов
def raw_to_indexes(lines: list[str]):
    indexes = torch.zeros((len(lines), MAX_LEN), dtype=int)
    for i in range(len(lines)):
        if raw_lines[i]:
            for j, w in enumerate(lines[i]):
                if j >= MAX_LEN: break
                indexes[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    return indexes

In [5]:
# обучающие данные
MAX_LEN = 30

# список всех строк
with open('datasets/onegin.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

### Полученные тензоры данных

In [6]:
raw_lines[0:3]

['мой дядя самых честных правил',
 'когда не в шутку занемог',
 'он уважать себя заставил']

In [7]:
encrypted_raw_lines[0:3]

['орл жбжб увоэч щзуфпэч ствдкн',
 'рфйкж ул и ющшрщ нжултфй',
 'хф ъйзнзщг шмиж озшщзйпт']

In [8]:
indexes[0:3]

tensor([[13, 15, 10, 33,  5, 32,  5, 32, 33, 18,  1, 13, 28, 22, 33, 24,  6, 18,
         19, 14, 28, 22, 33, 16, 17,  1,  3,  9, 12,  0],
        [11, 15,  4,  5,  1, 33, 14,  6, 33,  3, 33, 25, 20, 19, 11, 20, 33,  8,
          1, 14,  6, 13, 15,  4,  0,  0,  0,  0,  0,  0],
        [15, 14, 33, 20,  3,  1,  7,  1, 19, 29, 33, 18,  6,  2, 32, 33,  8,  1,
         18, 19,  1,  3,  9, 12,  0,  0,  0,  0,  0,  0]])

In [9]:
encrypted_indexes[0:3]

tensor([[15, 17, 12, 33,  7,  2,  7,  2, 33, 20,  3, 15, 30, 24, 33, 26,  8, 20,
         21, 16, 30, 24, 33, 18, 19,  3,  5, 11, 14,  0],
        [17, 21, 10, 11,  7, 33, 20, 12, 33,  9, 33, 31, 26, 25, 17, 26, 33, 14,
          7, 20, 12, 19, 21, 10,  0,  0,  0,  0,  0,  0],
        [22, 21, 33, 27, 10,  8, 14,  8, 26,  4, 33, 25, 13,  9,  7, 33, 15,  8,
         25, 26,  8, 10, 16, 19,  0,  0,  0,  0,  0,  0]])

In [10]:
indexes.shape, encrypted_indexes.shape # 1452 строки, длиной MAX_LEN символов

(torch.Size([1452, 30]), torch.Size([1452, 30]))

### Создание и обучение модели 

In [11]:
class Decryptor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decryptor, self).__init__()
        self.embedding = nn.Embedding(input_size, 64)
        self.rnn = nn.GRU(64, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.rnn(embedded)
        output = self.linear(output)
        return output

In [12]:
input_size = len(CHAR_TO_INDEX)
hidden_size = 256
output_size = len(CHAR_TO_INDEX)

### <проверка размерностей>

In [13]:
embedding = nn.Embedding(input_size, 64)
rnn = nn.GRU(64, hidden_size, batch_first=True)
linear = nn.Linear(hidden_size, output_size)

In [14]:
e = embedding(indexes[0:3])
e.shape

torch.Size([3, 30, 64])

In [15]:
r = rnn(e)[0]
r.shape

torch.Size([3, 30, 256])

In [16]:
lin = linear(r)
lin.shape

torch.Size([3, 30, 34])

### <проверка размерностей>

In [17]:
model = Decryptor(input_size, hidden_size, output_size)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
num_epochs = 200
for epoch in range(num_epochs):
    
    for i in range(int(len(indexes) /100)):
        X_batch = encrypted_indexes[i * 100 : (i + 1) * 100] # 100 x 30
        Y_batch = indexes[i * 100 : (i + 1) * 100] # 100 x 30
        Y_batch = Y_batch.flatten() # 3000
        
        optimizer.zero_grad()
        
        output = model(X_batch) # 100 x 30 x 64 ->  100 x 30 x 256 -> 100 x 30 x 34
        output = output.view(-1, output_size) # 3000 x 34
        
        loss = criterion(output, Y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 10, Loss: 1.17266047000885
Epoch: 20, Loss: 0.7597644329071045
Epoch: 30, Loss: 0.5062811970710754
Epoch: 40, Loss: 0.33038854598999023
Epoch: 50, Loss: 0.3306637108325958
Epoch: 60, Loss: 0.2194177806377411
Epoch: 70, Loss: 0.14344286918640137
Epoch: 80, Loss: 0.12009724974632263
Epoch: 90, Loss: 0.1056441143155098
Epoch: 100, Loss: 0.09641613066196442
Epoch: 110, Loss: 0.0903274416923523
Epoch: 120, Loss: 0.08618305623531342
Epoch: 130, Loss: 0.08327937871217728
Epoch: 140, Loss: 0.08120624721050262
Epoch: 150, Loss: 0.07968523353338242
Epoch: 160, Loss: 0.0785425528883934
Epoch: 170, Loss: 0.07766758650541306
Epoch: 180, Loss: 0.07698649913072586
Epoch: 190, Loss: 0.07644901424646378
Epoch: 200, Loss: 0.07602030783891678


### Тестирование данных (обучающая выборка)

In [20]:
with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)

In [21]:
decrypted_strings[0:20]

['иой дядя самых честных правил',
 'иогда не в шутку занемог',
 'по уважать себя заставил',
 'к лучше выдумать не мог',
 'иго пример другим наука',
 'по боже мой какая скука',
 'п больным сидеть и день и ночь',
 'не отходя ни шагу прочь',
 'какое низкое коварство',
 'полуживого забавлять',
 'иму подушки поправлять',
 'пачально подносить лекарство',
 'вздыхать и думать про себя',
 'иогда же черт возьмет тебя',
 '',
 'так думал молодой повеса',
 'иетя в пыли на почтовых',
 'всевышней волею зевеса',
 'наследник всех своих родных',
 'иосзья людмилы и руслана']

### Тестирование данных (тестовая выборка - onegin_test)

In [22]:
# список всех строк
with open('datasets/onegin_test.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)
    
decrypted_strings[20:40]

['пу что ж ты едешь очень жаль',
 'ич уочшео тлучсоп ке улрвне св',
 'увидеть мне филлиду эту',
 'поедлет з мыслей и пера',
 'и сеез и рифм  ',
 'представь меня  съ чттичь  нет',
 ' ч пад  когда же  хоть сейчас',
 'пои с очотой примут нас',
 'иоедем  поскакали други',
 'вемомуа ло твуурчены',
 'пнрой тяжелые услуги',
 'вортеопиилной старины',
 'иакые известный угошенья',
 'иесут ма блюдецках варенья',
 'на ттолик ставят вощаной',
 'вувшин с брусничною водой',
 'пои дорогой самой краткой',
 'вомой метят во весь ооор ',
 'накалы моврлтчадл ткоюдиои',
 'вероев наших разговор']